In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msn

C:\Users\Acer\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# ML Models
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ML Pre processing
from surprise.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Hyperparameter tuning
from surprise.model_selection import GridSearchCV

In [3]:
df_ratings = pd.read_table("dataSet/user_ratedmovies.dat",sep="\t")

In [4]:
df_ratings.head(2)

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44


In [73]:
df_temp = df_ratings[['userID','movieID','rating']]
df_temp.to_csv('modified_rating_dataSet.csv',index=False)

In [54]:
df_movies = pd.read_table("dataSet/movies.dat",sep="\t")
pd.set_option('display.max_columns',None)

In [55]:
df_movies.rename(columns={'id': 'movieID'}, inplace=True)

In [56]:
df_movies.head(2)

,movieID,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,rtAllCriticsNumRotten,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,0,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,15,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...


In [74]:
df_movies.isnull().sum()

movieID                     0
title                       0
imdbID                      0
spanishTitle                0
imdbPictureURL            181
year                        0
rtID                      311
rtAllCriticsRating          0
rtAllCriticsNumReviews      0
rtAllCriticsNumFresh        0
rtAllCriticsNumRotten       0
rtAllCriticsScore           0
rtTopCriticsRating          0
rtTopCriticsNumReviews      0
rtTopCriticsNumFresh        0
rtTopCriticsNumRotten       0
rtTopCriticsScore           0
rtAudienceRating            0
rtAudienceNumRatings        0
rtAudienceScore             0
rtPictureURL                0
dtype: int64

In [76]:
df_temp2 = df_movies[['movieID','title']]
#df_temp2.to_csv('modified_movies_dataSet.csv',index=False)

In [86]:
df_temp2.head(2)

,movieID,title
0,1,Toy story
1,2,Jumanji


In [77]:
is_duplicate = df_temp2.duplicated()

In [78]:
print("Number of duplicate records:", sum(is_duplicate))

Number of duplicate records: 0


In [79]:
df_temp2.to_csv('modified_movies_dataSet.csv',index=False)

In [5]:
df = df_ratings[['userID','movieID','rating']]

In [25]:
df.head(4)

,userID,movieID,rating
0,75,3,1.0
1,75,32,4.5
2,75,110,4.0
3,75,160,2.0


In [6]:
df.isnull().sum()

userID     0
movieID    0
rating     0
dtype: int64

In [7]:
is_duplicate = df.duplicated()

In [8]:
print("Number of duplicate records:", sum(is_duplicate))

Number of duplicate records: 0


In [9]:
# surprise reader API to read the dataset
reader = Reader()

In [12]:
data = Dataset.load_from_df(df_ratings[['userID','movieID','rating']], reader)
#data.split(n_folds=5)

In [18]:
algo = SVD()

In [16]:
# Data split 9/1
trainset, testset = train_test_split(data, test_size=0.01)

In [19]:
# Fitting our trainset
algo.fit(trainset)

In [20]:
predictions = algo.test(testset) 

In [44]:
accuracy.rmse(predictions)

RMSE: 0.7528


0.7528406875593947

In [21]:
test = pd.DataFrame(predictions)

In [22]:
test

,uid,iid,r_ui,est,details
0,64411,1219,5.0,4.090862,{'was_impossible': False}
1,5085,3262,2.5,3.317922,{'was_impossible': False}
2,39935,48082,5.0,4.549616,{'was_impossible': False}
3,48717,3072,0.5,3.159473,{'was_impossible': False}
4,20492,46948,3.5,3.803004,{'was_impossible': False}
...,...,...,...,...,...
8551,14918,5991,4.5,3.466088,{'was_impossible': False}
8552,28048,1240,3.5,3.835151,{'was_impossible': False}
8553,29658,1136,4.5,4.411280,{'was_impossible': False}
8554,36538,4639,3.0,2.905185,{'was_impossible': False}


In [80]:
import pickle
#open a file, where you ant to store the data
file = open('SVD_modelv2.pkl', 'wb')
# dump information to that file
pickle.dump(algo, file)
file.close()

In [98]:
x = df[df['userID'] == 1].shape

In [100]:
x[0]

0

In [62]:
df['userID'].unique()

array([   75,    78,   127, ..., 71525, 71529, 71534], dtype=int64)

In [95]:
algo.predict(1, 5107)

Prediction(uid=1, iid=5107, r_ui=None, est=3.1535657489816336, details={'was_impossible': False})

In [46]:
def get_recommendations(uid):
    prediction_list = []
    for mid in df_ratings['movieID'].unique():
        predic = algo.predict(uid,mid)
        prediction_list.append(predic)
    return prediction_list  

In [64]:
prediction = get_recommendations(127)

In [65]:
prediction[:5]

[Prediction(uid=127, iid=3, r_ui=None, est=3.6702614103262228, details={'was_impossible': False}),
 Prediction(uid=127, iid=32, r_ui=None, est=4.500144566275445, details={'was_impossible': False}),
 Prediction(uid=127, iid=110, r_ui=None, est=4.610423885799751, details={'was_impossible': False}),
 Prediction(uid=127, iid=160, r_ui=None, est=3.1152990389689514, details={'was_impossible': False}),
 Prediction(uid=127, iid=163, r_ui=None, est=3.70113909200238, details={'was_impossible': False})]

In [66]:
df_pred=pd.DataFrame(prediction)

In [67]:
# Renaming our predictions to original names
df_pred=df_pred.rename(columns={'uid':'userID', 'iid':'movieID','est':'rating'}).sort_values('rating',ascending=False).head(10)

In [68]:
df_pred

,userID,movieID,r_ui,rating,details
7,127,296,None,5.000000,{'was_impossible': False}
2200,127,63082,None,4.940701,{'was_impossible': False}
276,127,2542,None,4.934788,{'was_impossible': False}
550,127,50,None,4.912259,{'was_impossible': False}
1622,127,7767,None,4.886759,{'was_impossible': False}
6199,127,2931,None,4.879091,{'was_impossible': False}
258,127,2360,None,4.867947,{'was_impossible': False}
629,127,1213,None,4.866306,{'was_impossible': False}
953,127,2351,None,4.857233,{'was_impossible': False}
34,127,2959,None,4.847372,{'was_impossible': False}


In [69]:
df_final = df_pred.merge(df_movies ,on='movieID',how='left')

In [70]:
df_final.head(3)

,userID,movieID,r_ui,rating,details,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,rtAllCriticsNumRotten,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,127,296,None,5.000000,{'was_impossible': False},Pulp Fiction,110912,Pulp Fiction,http://ia.media-imdb.com/images/M/MV5BMjE0ODk2...,1994,pulp_fiction,8.9,50,47,3,94,8.4,14,13,1,92,4.2,249269,95,http://content9.flixster.com/movie/10/88/65/10...
1,127,63082,None,4.940701,{'was_impossible': False},Slumdog Millionaire,1010048,Slumdog Millionaire,http://ia.media-imdb.com/images/M/MV5BMTU2NTA5...,2008,slumdog_millionaire,8.2,220,206,14,93,8.2,38,36,2,94,4,328969,90,http://content9.flixster.com/movie/10/86/47/10...
2,127,2542,None,4.934788,{'was_impossible': False},"Lock, Stock and Two Smoking Barrels",120735,Lock & Stock,http://ia.media-imdb.com/images/M/MV5BMTM5MTk2...,1998,NaN,6.8,51,37,14,72,5.9,14,8,6,57,4,81696,92,http://content6.flixster.com/movie/10/51/33/10...


In [71]:
movies_list = df_final['title'].values

In [61]:
movies_list

array(['Le notti di Cabiria', 'Das Leben der Anderen', 'La strada',
       '28 Up', '3 Women',
       'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb',
       'Matewan', 'Tystnaden', 'Wallace & Gromit in The Wrong Trousers',
       'Persona'], dtype=object)

In [72]:
movies_list

array(['Pulp Fiction', 'Slumdog Millionaire',
       'Lock, Stock and Two Smoking Barrels', 'The Usual Suspects',
       'La meglio giovent�', 'Dom za vesanje', 'Festen', 'Goodfellas',
       'Le notti di Cabiria', 'Fight Club'], dtype=object)